* TECH CHALLENGE - FASE 03: FINE-TUNING AMAZON PRODUCTS
* Versão baseada no notebook do professor (Unsloth)

* 1: CONFIGURAÇÃO E INSTALAÇÃO DAS DEPENDÊNCIAS

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets gradio

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
import gradio as gr

In [ ]:
print(f"PyTorch versão: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")

* 2: PREPARAÇÃO DO DATASET AMAZON

In [ ]:
!gunzip trn.json.gz

In [ ]:
try:
    with open('/content/trn.json', 'r') as f:
        test_line = f.readline()
    print("Dataset encontrado!")
except FileNotFoundError:
    print("Erro ao tentar carregar o arquivo!")

* 3: FORMATAÇÃO DOS DADOS PARA FINE-TUNING

In [ ]:
def format_amazon_dataset_expanded(json_path="/content/trn.json", max_examples=150):

    print(f"Formatting {max_examples} fine-tuning products...")

    training_examples = []

    with open(json_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_examples:
                break

            try:
                item = json.loads(line)
                title = item.get('title', '').strip()
                content = item.get('content', '').strip()

                if not title or not content:
                    continue

                # MAIS variações de perguntas para melhor generalização
                questions = [
                    f"Tell me about this product: {title}",
                    f"What are the characteristics of: {title}",
                    f"Describe in detail: {title}",
                    f"What can you tell me about: {title}",
                    f"I need information about: {title}"
                ]

                # 3 exemplos por produto
                for question in questions[:3]:
                    example = {
                        "instruction": "DESCRIBE THIS AMAZON PRODUCT",
                        "input": question,
                        "output": content
                    }
                    training_examples.append(example)

            except json.JSONDecodeError:
                continue

    print(f"Expanded dataset: {len(training_examples)} created examples")
    return training_examples

In [ ]:
amazon_training_data = format_amazon_dataset_expanded(max_examples=200)

In [ ]:
with open('/content/amazon_formatted_data_expanded.json', 'w', encoding='utf-8') as f:
    json.dump(amazon_training_data, f, ensure_ascii=False, indent=2)

print(f"Expanded data saved: {len(amazon_training_data)} examples")
print(f"Example:")
print(json.dumps(amazon_training_data[0], indent=2, ensure_ascii=False))

* 4: CONFIGURAÇÃO DO MODELO BASE

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

print("Loading Llama-3-8B model...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("Model loaded!")
print(f"Parameters: {model.num_parameters():,}")

* 5: TESTE DO MODELO ANTES DO FINE-TUNING

In [ ]:
FastLanguageModel.for_inference(model)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Teste simples que o modelo base consegue processar
test_input_simple = "Describe Sony WH-1000XM4 headphones:"

# Modified to pass input as a list
inputs = tokenizer([test_input_simple], return_tensors="pt").to("cuda") # return_tensors="pt" pytorch

print("Testing BASE model...")
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.batch_decode(outputs)[0]
print(response.replace("<|begin_of_text|>", ""))

* 6: CONFIGURAÇÃO DO FINE-TUNING COM LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("LoRA configured!")
print(f"Trainable parameters: {model.print_trainable_parameters()}")

* 7: PREPARAÇÃO DOS DADOS PARA TREINAMENTO

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = Dataset.from_list(amazon_training_data)
print(f"Dataset: {len(dataset)} examples")

dataset = dataset.map(formatting_prompts_func, batched=True)
print("Formatted dataset!")

* 8: CONFIGURAÇÃO E EXECUÇÃO DO TREINAMENTO

In [ ]:
print("Setting up expanded training...")

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=15,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

print("Starting expanded training...")

In [ ]:
trainer_stats = trainer.train()

print("Training completed!")
print(f"Final Loss: {trainer_stats.training_loss:.4f}")

* 9: TESTE DO MODELO APÓS FINE-TUNING

In [ ]:
FastLanguageModel.for_inference(model)

test_questions = [
    "Describe Sony WH-1000XM4 headphones",
    "What are the features of: Samsung Galaxy S21 Ultra",
    "What can you tell me about: Nike Air Max 270"
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"QUESTION: {question}")

    test_prompt = alpaca_prompt.format(
        "DESCRIBE THIS AMAZON PRODUCT",
        question,
        ""
    )

    inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")

    print("RESPONSE AFTER FINE-TUNING:")
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)

    outputs = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(full_response)

* 10: SALVAR MODELO

In [ ]:
print("Salvando modelo expandido...")

model.save_pretrained("amazon_product_model_expanded")
tokenizer.save_pretrained("amazon_product_model_expanded")

* 11: EXECUÇÃO MANUAL MODELO

In [ ]:
test_prompt = alpaca_prompt.format(
    "DESCRIBE THIS AMAZON PRODUCT",
    "Tell me about: Nike Air Max 270",
    ""
)

inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")

print("RESPONSE AFTER FINE-TUNING:")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

outputs = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=150,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(full_response)